# 필요 모듈 임포트

In [207]:
import re
import pandas as pd
import numpy as np
from konlpy.tag import Kkma
import re
import unicodedata
import re
import nltk
from string import whitespace, punctuation
from nltk.tokenize import sent_tokenize
from hanspell import spell_checker
from pykospacing import Spacing
from tqdm import tqdm
from kss import split_sentences
import warnings

tqdm.pandas()
warnings.filterwarnings('ignore')

In [208]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/parkjunhyeong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 데이터 로드

In [209]:
#기간별 SNS 데이터 로드 및 병합
load_path = './'
comm_df1 = pd.read_excel(load_path + 'Analysis_SnsData_20190501_20210930_(청년정책)&&~(rt__독취사)_BY_ARGOS.xlsx', sheet_name = '커뮤니티', skiprows = 1)
comm_df2 = pd.read_excel(load_path + 'Analysis_SnsData_20211001_20230626_(청년정책)&&~(rt__독취사)_BY_ARGOS.xlsx', sheet_name = '커뮤니티', skiprows = 1)
comm_df = pd.concat([comm_df1, comm_df2], ignore_index = True)

In [210]:
comm_df

,작성일,작성자,제목,내용
0,2021/01/12,스펙업,2021 청년정책 총정리! 모르면 나만손해임! (+ 1월부터 시작되는 국민취업지원제...,\r\n \r\n \r\n \r\n \r\n \r\n2021 청년정책! 얼마...
1,2020/12/15,스펙업,온라인 청년센터 청년정책 한 눈에 알아보기! 2021에 새로 시작되는 취업지원 제도는?,\r\n \r\n \r\n \r\n \r\n \r\n대학생/취준생/2~30대...
2,2020/10/29,스펙업,청년들만 누릴 수 있는 혜택 빵빵!! 금융 관련 청년정책 모아봤습니다:D,\r\n \r\n \r\n \r\n \r\n \r\n청년들만 누릴 수 있는 ...
3,2021/04/07,취업대학교,2021 청년정책 놓치지 않을거에요! 온라인 청년센터 100% 활용하기!,\r\n \r\n \r\n \r\n \r\n \r\n \r\n ...
4,2021/02/05,스펙업,경기도 청년정책 리스트 확인해보시고 청년지원금 받아가세용 !!,\r\n \r\n \r\n \r\n \r\n \r\n경기도 청년정책 !!! ...
...,...,...,...,...
4251,2021/10/07,거제도 거사모,거제시 청년 월세 지원,https://www.geoje.go.kr/yesan/index.geoje?menu...
4252,2021/10/06,스펙업,"대학생, 취준생을 위한 정책이 은근 많네요! 서로 정보 공유해요~!","​ \r\n대학생, 취준생을 위한 청년정책이 은근 많네요 \r\n​ \r\n취준생들..."
4253,2021/10/06,MLB파크,윤캠 국힘갤 모니터링 이분이 한듯,이분이 커뮤여론 모니터링도 하고 청년정책 짜는 모양이네요
4254,2021/10/05,디시인사이드,난 ㄹㅇ 이재명 뽑을거임,어쩌피 집값이랑 출산율은 누가되도 씹창이고 \r\n이재명 청년 정책이 나한테 가장 ...


# 전처리

In [211]:
#중복 제거
comm_df_n = comm_df.copy()
comm_df_n.drop_duplicates(subset = '내용', inplace = True, ignore_index = True)
comm_df_n.drop_duplicates(subset = '제목', inplace = True, ignore_index = True)
comm_df_n.dropna(inplace = True)
comm_df_n.reset_index(inplace = True, drop = True)

In [212]:
#광고성 게시글 제거
#TBD: 고도화 여지 있음
comm_df_n = comm_df_n[~(comm_df_n['내용'].str.contains('http'))]
comm_df_n = comm_df_n[~(comm_df_n['내용'].str.contains('.com'))]
removes = ['스펙업', '팍스넷', '아름다운 귤현동 사람들']
comm_df_n = comm_df_n[~(comm_df_n['작성자'].isin(removes))].reset_index(drop=True)
comm_df_n

,작성일,작성자,제목,내용
0,2021/08/27,카페파우더룸,"\""이런 정책이 있었어?\"" 놓치면 후회할 청년정책",대한민국 청년들을 위한 정책 모음! \r\n​ \r\n정책주간지 <공감>이 알아두면...
1,2021/09/25,에펨코리아,진짜 청년정책은 청년타이틀부터 빼는게 청년정책이다,청년정책이라고 내놓는거 보면\r\n지돈 아니라고 청년주택 청년대출 청년보조금 청년할...
2,2019/05/02,클리앙,"당정청, 20대男 민심잡기…청년정책 ‘컨트롤타워’ 신설","더불어민주당과 정부, 청와대는 2일 청년 정책을 종합적이고 체계적으로 추진하기 위해..."
3,2021/09/25,에펨코리아,홍준표가 청년정책이 왜없냐?,이게 청년정책임.\r\n미래를 살아갈 우리에게 새로운 일자리와 국가의 성장동력을 주...
4,2021/09/15,울산부동산재테크,"(경상일보, 울산시) 청년정책 아이디어 PT 경연대회 참가자 모집",울산 청년이 생각하는 청년정책 아이디어 경연대회를 개최합니다.~~ \r\n청년들의 ...
...,...,...,...,...
1542,2021/10/12,딴지일보,진짜 뭔가 감개무량하고 꿈같아요 이재명 후보님 최종확정이요,성남시에서 청년정책으로 기본소득 혜택 받았거든요 그땐 거기에 살았어서 \r\n성남...
1543,2021/10/09,에펨코리아,이 정부는 제발 아무것도 안하고 가만히 있으면 좋겠다,그냥 무정부상태 처럼 가만히ㅋㅋ \r\n문정부상태가 더 무섭다 \r\n어떻게 청년정...
1544,2021/10/08,에펨코리아,함슬옹은 진짜 억까지 밭갈이 시발련들아,예전에 준스톤욕했다고 그러는거야?\r\n나중에 홍카가 경선1위하고 국힘애들 집합할때...
1545,2021/10/06,MLB파크,윤캠 국힘갤 모니터링 이분이 한듯,이분이 커뮤여론 모니터링도 하고 청년정책 짜는 모양이네요


In [213]:
#전처리 함수

def clean_str1(text, special = False):
    #E-mail 제거
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    #. ? ! 등의 반복 제거
    pattern = r'\.+'
    text = re.sub(pattern=pattern, repl='.', string=text)
    pattern = r'\?+'
    text = re.sub(pattern=pattern, repl='?', string=text)
    pattern = r'\!+'
    text = re.sub(pattern=pattern, repl='!', string=text)
    pattern = r'\!~'
    text = re.sub(pattern=pattern, repl='~', string=text)
    pattern = r'\!,'
    text = re.sub(pattern=pattern, repl=',', string=text)
    # 한글 자음, 모음 제거
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # HTML 태그 제거
    pattern = '<[^>]*>'        
    text = re.sub(pattern=pattern, repl='', string=text)
    # 특수기호 제거
    if special:
        pattern = '[^\w\s\n]'
        text = re.sub(pattern=pattern, repl='', string=text)
        text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', string=text)
    
    return text.strip()

def clean_str2(text):
    text = text.replace(u'\xa0',u'')
    text = text.replace(u'\u200b',u'')
    text = text.replace(u'\uD83C',u'')
    text = text.replace(u'\uDF1F',u'')
    text = text.replace(u'\u3000',u'')
    text = text.replace(u'\u200d',u'')
    text = text.replace(u'\u2002',u'')
    text = text.replace(u'\u2008',u'')
    text = text.replace(u'\uD83D',u'')
    text = text.replace(u'\uDCB4',u'')
    text = text.replace(u'\uDCCC',u'')
    text = text.replace(u'\uDCCA',u'')
    text = text.replace(u'\uDCE2',u'')
    return text.strip()

In [214]:
# 문장 단위 semantic을 기준으로 임베딩할 것이므로 \n \t 등을 모두 공백으로 처리
pattern_whitespace = re.compile(f'[{whitespace}]+')

# sns 제목 공백 처리 
comm_df_n['제목_n'] = comm_df_n['제목'].str.replace(pattern_whitespace, ' ')
comm_df_n['제목_n'] = comm_df_n['제목_n'].apply(lambda x : unicodedata.normalize('NFC', x)).str.strip()
# tbd: 제목에서 유니코드 공백 문자 제거 > 방법 더 연구해봐야 함
# comm_df_n['제목_n'] = comm_df_n['제목_n'].apply(lambda x : x.encode('cp949', 'ignore').decode('cp949'))

# sns 본문 공백 처리
comm_df_n['내용_n1'] = comm_df_n['내용'].str.replace(pattern_whitespace, ' ')
comm_df_n['내용_n1'] = comm_df_n['내용_n1'].apply(lambda x : unicodedata.normalize('NFC', x)).str.strip()

In [215]:
#version1 : 1) sns 데이터 전처리, 2) 문장 분리

#전처리
comm_df_n['내용_n1'] = comm_df_n['내용_n1'].apply(lambda x : clean_str1(x,special = False))

#문장 분리
comm_df_n['내용_n1'] = comm_df_n['내용_n1'].progress_apply(lambda x: [clean_str2(s) for s in split_sentences(x)])

100%|███████████████████████████████████████| 1547/1547 [00:21<00:00, 72.48it/s]


In [216]:
#version2 : 1) sns 데이터 전처리, 2) 문장 분리, 3)분리된 문장의 특수 문자 제거

#문장 분리 후 특수 문자 완전 제거
comm_df_n['내용_n2'] = comm_df_n['내용_n1'].apply(lambda x : [clean_str1(sent,special = True) for sent in x])

# 전처리 결과 저장

In [217]:
store_path = './'
comm_df_n.to_excel(store_path + 'community_preprocessed_v0.1.xlsx')